In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
options = Options() 
chrome_path = r'\Users\User\Documents\chromedriver'

In [3]:
options.add_argument('--window-size=500,300')

In [4]:
driver = webdriver.Chrome(chrome_path, 
                              options=options)

driver.set_window_size(1400,1000)

In [5]:
driver.get('http://101.78.205.36/racing/information/English/Horse/OtherHorse.aspx?HorseId=HK_2017_B233')
time.sleep(7)

In [6]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)
tables = soup.find('table', attrs = {'class' : 'bigborder'})
table = tables.prettify()
df = pd.read_html(table, header = 0)[0]

In [15]:
race_hist = df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,
                     33,34,35])

In [16]:
g = race_hist

In [18]:
def frmt(df):
    global name
    name = df.loc[1][0]

In [19]:
def get_race_hist(df):
    try:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
        tables = soup.find('table', attrs = {'class' : 'bigborder'})
        table = tables.prettify()
        df = pd.read_html(table, header = 0)[0]
        df.dropna(how='all', inplace = True)
        df.reset_index(drop = True, inplace = True)
        prcs(df)
    except:
        return 'nothin'

In [20]:
def prcs(df):
    plcmnt = df['Pla.'].values
    tdrop = []
    for i, val in enumerate(plcmnt):
        if val.endswith('Season'):
            tdrop.append(i)
    df.drop(tdrop, inplace = True)
    df['hname'] = name
    a = df.loc[[1]]
    global g
    g = pd.concat([g,a], ignore_index = True, sort = True)

In [21]:

races = ['2','3','4','5','6','7','8','9','10','11','12']
rows = ['1','2','3','4','5','6','7','8','9','10','11','12',
        '13','14','15','16','17']
for race in races:
    for r in rows:
        try:
            time.sleep(3)
            driver.get('http://101.78.205.36/racing/information/English/racing/LocalResults.aspx')
            driver.find_element_by_xpath('/html/body/div/div[2]/table/tbody/tr/td[' + race + ']').click()
            driver.find_element_by_xpath('/html/body/div/div[5]/table/tbody/tr[' + r + ']/td[3]/a').click()
            page_source = driver.page_source
            soup = BeautifulSoup(page_source)
            tables = soup.findAll('table')
            table = tables[0].prettify()
            df = pd.read_html(table)[0]
            frmt(df)
            get_race_hist(df)

        except:
            continue



In [24]:
dates = []
retiredhorse = 0
for date in g['Date'].values:
    if date.endswith('2020'):
        date = date[:6] + date[8:]
        dates.append(date)
        retiredhorse += 1
    else:
        dates.append(date)

In [25]:
g['Date'] = dates

In [27]:
g.to_csv('update_races.csv', index = False)

In [28]:
driver.quit()

In [29]:
if retiredhorse > 0:
    g['RC/Track/  Course'] = g['RC/Track/  Course'].replace(np.nan,'')
    g['RC  /Track/  Course'] = g['RC  /Track/  Course'].replace(np.nan,'')
    g['RC  /Track/  Course'] = g['RC  /Track/  Course'] + g['RC/Track/  Course']

In [31]:
rctc = g['RC  /Track/  Course'].values

In [32]:
RC = []
Track = []
Course = []

for row in rctc:
    x = row.split(' / ')
    if len(x) == 2:
        Course.append(np.nan)
    for i, cond in enumerate(x):
        cond = cond.replace('"', '')
        if i == 0:
            RC.append(cond)
        elif i == 1:
            Track.append(cond)
        elif i == 2:
            Course.append(cond)
        else:
            continue

In [33]:
g['RC'] = RC
g['Track'] = Track
g['Course'] = Course

In [34]:
g.drop('RC  /Track/  Course', axis = 1, inplace = True)

In [44]:
col_len = len(g.columns)

In [46]:
if col_len > 22:
    g.drop('RC/Track/  Course', axis = 1, inplace = True)

In [47]:
names = g['hname'].values

In [48]:
hnames = []
for name in names:
    x = name.split(' (')
    for i in x:
        if i.endswith(')'):
            continue
        else:
            hnames.append(i)

In [49]:
g['hname'] = hnames

In [50]:
g['U_raceno'] = g['Race  Index'] + ' ' + g['Date']

In [51]:
g.drop('Video  Replay', axis = 1, inplace = True)
g.drop('Rtg.', axis = 1, inplace = True)
g.drop('Gear', axis = 1, inplace = True)
g.drop('G', axis = 1, inplace = True)

In [1]:
rh = pd.read_csv('race_hist.csv')

In [54]:
# update the races file with the new races
new_rh = pd.concat([rh, g], ignore_index = True, sort = True)

In [55]:
new_rh.to_csv('race_hist.csv', index = False)